In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading dataset from the previous lab (people identified as donors, including false donors)
# Important note: this potential donors only correspond to people identified as donors in the test dataset.
predicted_donors = pd.read_csv('predicted_donors.csv')

In [3]:
predicted_donors.head()

,Unnamed: 0,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,y_pred,TARGET_D
0,0,28,65.000000,6,9,0,0,24,42,22,9,2,1,9,10575,3049,4490,98,0,2,47,53,90,2,1,4,9,0,1,1,1,0,0,0,5,0,1,2,35,42,45,39,47,50,20,47,36,17,9,25,19,12,11,13,12,22,18,31,17,12,33,15,31,26,40,33,18,6,2,63,10,9,18,159,235,65,34,1,27,26,12,0,0,0,1953,2155,8,8,87,13,93,7,2,27,68,59,23,82,18,4,1,3,14,23,6,15,52,23,1,1,0,47,75,89,94,99,10,2,40,51,53,13,4,25,2,7,4,2,6,1,88,90,93,94,4480.0,13.0,803.0,454,526,478,564,21667,15,13,12,16,28,9,2,2,2,4,11,11,18,35,12,4,2,2,33,3,52,4,61,39,7,80,14,1,1,0,0,1,1,3,0,3,33,37,15,72,64,76,54,72,51,64,45,98,79,2,17,18,4,16,19,0,4,5,1,9,3,1,3,2,0,5,23,2,4,5,14,11,5,1,5,6,5,7,4,10,2,1,8,75,5,0,138,2,9,24,33,10,17,5,4,18,2,13,7,0,0,0,16,32,2,18,23,45,8,0,6,1,8,0,0,4,4,1,2,0,2,1,1,0,11,47,88,6,3,4,96,61,17,4,13,4,34,44,65,98,2,48,3,82,2,15,0,0,1,98,2,95,99,99,13,3,5,6,22,53,6,13,81.0,8,5,3.0,17.0,17.0,1,10.125000,148993,1,1,11,22,2,1,91,1,32,8,91,10,95,11,95,11,91,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
1,3,0,61.611649,5,9,0,0,33,37,49,8,8,4,2,2540,732,821,99,0,0,51,49,99,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,33,42,45,33,43,46,30,30,45,25,12,18,24,23,14,6,3,13,12,33,24,18,13,3,12,9,30,61,38,15,4,70,6,3,21,247,309,94,92,5,5,0,0,0,0,0,961,1071,5,4,90,10,98,2,0,49,89,80,43,95,5,6,1,5,7,7,2,24,65,10,0,0,0,4,16,46,74,98,0,0,77,65,67,17,4,5,0,0,6,4,0,0,32,57,83,94,7040.0,75.0,609.0,493,500,546,550,17749,4,3,12,34,33,9,2,1,2,2,2,15,30,35,10,3,1,1,13,2,63,1,44,56,14,89,8,0,0,0,0,0,0,3,3,47,21,24,1,63,78,85,70,80,68,72,58,99,99,0,19,15,6,16,15,0,1,8,0,11,5,2,2,0,0,6,26,4,3,5,11,11,4,3,0,10,8,7,2,7,1,1,7,74,9,0,132,4,8,31,24,6,17,9,13,20,2,25,6,0,0,0,17,33,1,26,30,17,8,2,3,0,34,0,0,4,0,0,1,0,1,1,1,0,1,66,98,0,0,2,98,87,24,10,17,4,12,16,66,91,9,0,0,85,2,10,2,0,1,99,0,99,99,99,9,3,7,7,29,67,3,6,136.0,20,12,2.0,10.0,10.0,1,6.800000,30849,0,2,5,38,3,2,86,1,0,2,91,2,89,2,96,2,86,11,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0
2,6,2,61.611649,3,2,0,0,30,48,37,9,5,6,4,2603,7

In [4]:
# Number of potential donors, identified from the previous lab
predicted_donors.shape

(7453, 357)

In [6]:
# 6911 false donors (92.7%)!!
predicted_donors['TARGET_D'].value_counts()

0.0      6911
10.0      133
5.0        89
15.0       54
20.0       32
12.0       23
3.0        22
6.0        21
7.0        20
4.0        20
8.0        20
25.0       15
11.0       15
9.0        11
13.0       11
16.0        8
14.0        8
21.0        5
19.0        4
2.0         4
18.0        4
17.0        4
30.0        2
100.0       2
24.0        2
75.0        2
50.0        2
23.0        2
35.0        1
22.0        1
36.0        1
1.0         1
4.5         1
33.0        1
12.5        1
Name: TARGET_D, dtype: int64

In [7]:
# Quantity actually donated (what we are going to estimate with our model)
predicted_donors['TARGET_D'].sum()

6172.0

In [8]:
predicted_donors.drop(['Unnamed: 0', 'y_pred'], axis=1)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,TARGET_D
0,28,65.000000,6,9,0,0,24,42,22,9,2,1,9,10575,3049,4490,98,0,2,47,53,90,2,1,4,9,0,1,1,1,0,0,0,5,0,1,2,35,42,45,39,47,50,20,47,36,17,9,25,19,12,11,13,12,22,18,31,17,12,33,15,31,26,40,33,18,6,2,63,10,9,18,159,235,65,34,1,27,26,12,0,0,0,1953,2155,8,8,87,13,93,7,2,27,68,59,23,82,18,4,1,3,14,23,6,15,52,23,1,1,0,47,75,89,94,99,10,2,40,51,53,13,4,25,2,7,4,2,6,1,88,90,93,94,4480.0,13.0,803.0,454,526,478,564,21667,15,13,12,16,28,9,2,2,2,4,11,11,18,35,12,4,2,2,33,3,52,4,61,39,7,80,14,1,1,0,0,1,1,3,0,3,33,37,15,72,64,76,54,72,51,64,45,98,79,2,17,18,4,16,19,0,4,5,1,9,3,1,3,2,0,5,23,2,4,5,14,11,5,1,5,6,5,7,4,10,2,1,8,75,5,0,138,2,9,24,33,10,17,5,4,18,2,13,7,0,0,0,16,32,2,18,23,45,8,0,6,1,8,0,0,4,4,1,2,0,2,1,1,0,11,47,88,6,3,4,96,61,17,4,13,4,34,44,65,98,2,48,3,82,2,15,0,0,1,98,2,95,99,99,13,3,5,6,22,53,6,13,81.0,8,5,3.0,17.0,17.0,1,10.125000,148993,1,1,11,22,2,1,91,1,32,8,91,10,95,11,95,11,91,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,61.611649,5,9,0,0,33,37,49,8,8,4,2,2540,732,821,99,0,0,51,49,99,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,33,42,45,33,43,46,30,30,45,25,12,18,24,23,14,6,3,13,12,33,24,18,13,3,12,9,30,61,38,15,4,70,6,3,21,247,309,94,92,5,5,0,0,0,0,0,961,1071,5,4,90,10,98,2,0,49,89,80,43,95,5,6,1,5,7,7,2,24,65,10,0,0,0,4,16,46,74,98,0,0,77,65,67,17,4,5,0,0,6,4,0,0,32,57,83,94,7040.0,75.0,609.0,493,500,546,550,17749,4,3,12,34,33,9,2,1,2,2,2,15,30,35,10,3,1,1,13,2,63,1,44,56,14,89,8,0,0,0,0,0,0,3,3,47,21,24,1,63,78,85,70,80,68,72,58,99,99,0,19,15,6,16,15,0,1,8,0,11,5,2,2,0,0,6,26,4,3,5,11,11,4,3,0,10,8,7,2,7,1,1,7,74,9,0,132,4,8,31,24,6,17,9,13,20,2,25,6,0,0,0,17,33,1,26,30,17,8,2,3,0,34,0,0,4,0,0,1,0,1,1,1,0,1,66,98,0,0,2,98,87,24,10,17,4,12,16,66,91,9,0,0,85,2,10,2,0,1,99,0,99,99,99,9,3,7,7,29,67,3,6,136.0,20,12,2.0,10.0,10.0,1,6.800000,30849,0,2,5,38,3,2,86,1,0,2,91,2,89,2,96,2,86,11,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,61.611649,3,2,0,0,30,48,37,9,5,6,4,2603,723,971,0,0,99,51,49,97,0,1,0

In [9]:
# All data is numerical
predicted_donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7453 entries, 0 to 7452
Columns: 357 entries, Unnamed: 0 to TARGET_D
dtypes: float64(34), int64(323)
memory usage: 20.3 MB


In [10]:
y = predicted_donors['TARGET_D']
X = predicted_donors.drop(['TARGET_D'], axis = 1)

In [11]:
# No improvement found when using StandardScaler
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler().fit(X)
x_normalized = transformer.transform(X)

In [12]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(X)
x_normalized = transformer.transform(X)

In [13]:
# Train-test split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_normalized, y, test_size=0.3, random_state=0)

In [14]:
# Linear regression model
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [15]:
# Metrics for X_Train
from sklearn.metrics import r2_score
predictions_train = lm.predict(X_train)
predictions_train[predictions_train<0]=0  # removing negative values
r2_score(y_train, predictions_train)

0.10014293255585005

In [16]:
# Metrics for X_Test
predictions_test = lm.predict(X_test)
predictions_test[predictions_test<0]=0  # removing negative values
r2_score(y_test, predictions_test)

-0.06221304241711856

In [25]:
# Actual returns for the test dataset
y_test.sum()

1847.5

In [24]:
# Predicted returns for the test dataset
predictions_test.sum()

2196.3673555024598

In [17]:
predictions=np.concatenate((predictions_train, predictions_test))

In [18]:
len(predictions)

7453

In [22]:
# Quantity expected to collect from donors
round(sum(predictions),0)

7166.0

In [ ]:
# Very poor scores for this model (negative score for test set???)
# We are overestimating return ($7166 instead of $6172)
# The predicted profit would be $7166-7453*$0.68=$2097.96
# We must take into account that we are targeting only the test population from the previous lab, not the entire population.